## Import Modules

In [1]:
#import modules
from statsbombpy import sb

import pandas as pd
import matplotlib.pyplot as plt
import json

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier
from sklearn.inspection import permutation_importance

import numpy as np
import math
import time
import warnings
warnings.filterwarnings('ignore')

## Define specialized variables

In [ ]:
match_week_stage = math.floor((3/4)*34)
print(match_week_stage)

## Obtain League and shots data from Statsbomb

In [3]:
competitions = sb.competitions()

In [7]:
competitions[
    (competitions['competition_name'].isin([
    'Premier League', '1. Bundesliga', 'La Liga', 'Ligue 1', 'Serie A']))
             & 
    (competitions['season_name']=='2015/2016')
            ]

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-12-12T07:43:33.436182,None,None,2023-12-12T07:43:33.436182
41,11,27,Spain,La Liga,male,False,False,2015/2016,2023-09-20T17:21:32.111535,2021-06-13T16:17:31.694,None,2023-09-20T17:21:32.111535
58,7,27,France,Ligue 1,male,False,False,2015/2016,2023-12-13T00:27:57.162379,None,None,2023-12-13T00:27:57.162379
62,2,27,England,Premier League,male,False,False,2015/2016,2023-11-14T11:25:33.476498,2021-06-13T16:17:31.694,None,2023-11-14T11:25:33.476498
64,12,27,Italy,Serie A,male,False,False,2015/2016,2023-12-13T17:32:46.423081,None,None,2023-12-13T17:32:46.423081


In [9]:
Bundesliga = sb.matches(competition_id=9, season_id=27)
Bundesliga.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3890561,2016-05-14,15:30:00.000,Germany - 1. Bundesliga,2015/2016,Hoffenheim,Schalke 04,1,4,available,...,None,34,Regular Season,PreZero Arena,Felix Brych,Julian Nagelsmann,André Breitenreiter,1.1.0,2,2
1,3890505,2016-04-02,15:30:00.000,Germany - 1. Bundesliga,2015/2016,Bayern Munich,Eintracht Frankfurt,1,0,available,...,None,28,Regular Season,Allianz Arena,Florian Meyer,Josep Guardiola i Sala,Niko Kovač,1.1.0,2,2
2,3890511,2016-04-08,20:30:00.000,Germany - 1. Bundesliga,2015/2016,Hertha Berlin,Hannover 96,2,2,available,...,None,29,Regular Season,Olympiastadion Berlin,Benjamin Brand,Pál Dárdai,Daniel Stendel,1.1.0,2,2
3,3890515,2016-04-09,15:30:00.000,Germany - 1. Bundesliga,2015/2016,Hamburger SV,Darmstadt 98,1,2,available,...,None,29,Regular Season,Volksparkstadion,Peter Sippel,Bruno Labbadia,Dirk Schuster,1.1.0,2,2
4,3890411,2015-12-20,16:30:00.000,Germany - 1. Bundesliga,2015/2016,Hertha Berlin,FSV Mainz 05,2,0,available,...,None,17,Regular Season,Olympiastadion Berlin,Peter Sippel,Pál Dárdai,Martin Schmidt,1.1.0,2,2


### Get idea of column names for league matches

In [10]:
Bundesliga.columns

Index(['match_id', 'match_date', 'kick_off', 'competition', 'season',
       'home_team', 'away_team', 'home_score', 'away_score', 'match_status',
       'match_status_360', 'last_updated', 'last_updated_360', 'match_week',
       'competition_stage', 'stadium', 'referee', 'home_managers',
       'away_managers', 'data_version', 'shot_fidelity_version',
       'xy_fidelity_version'],
      dtype='object')

In [11]:
print(pd.unique(Bundesliga['home_team']))

['Hoffenheim' 'Bayern Munich' 'Hertha Berlin' 'Hamburger SV'
 'Borussia Dortmund' 'Wolfsburg' 'Schalke 04' 'FSV Mainz 05' 'Augsburg'
 'Bayer Leverkusen' 'Darmstadt 98' 'Werder Bremen' 'FC Köln'
 'Eintracht Frankfurt' 'VfB Stuttgart' 'Borussia Mönchengladbach'
 'Ingolstadt' 'Hannover 96']


In [14]:
example = sb.events(match_id=3754037)
example.columns

Index(['ball_receipt_outcome', 'ball_recovery_offensive',
       'ball_recovery_recovery_failure', 'carry_end_location',
       'clearance_aerial_won', 'clearance_body_part', 'clearance_head',
       'clearance_left_foot', 'clearance_right_foot', 'counterpress',
       'dribble_nutmeg', 'dribble_outcome', 'dribble_overrun', 'duel_outcome',
       'duel_type', 'duration', 'foul_committed_advantage',
       'foul_committed_card', 'foul_committed_offensive',
       'foul_committed_type', 'foul_won_advantage', 'foul_won_defensive',
       'goalkeeper_body_part', 'goalkeeper_end_location', 'goalkeeper_outcome',
       'goalkeeper_position', 'goalkeeper_technique', 'goalkeeper_type', 'id',
       'index', 'interception_outcome', 'location', 'match_id', 'minute',
       'miscontrol_aerial_won', 'off_camera', 'out', 'pass_aerial_won',
       'pass_angle', 'pass_assisted_shot_id', 'pass_body_part', 'pass_cross',
       'pass_cut_back', 'pass_end_location', 'pass_goal_assist', 'pass_height',
   

In [15]:
relevant_attributes = ['minute', 'second', 'team', 'shot_statsbomb_xg',
                       'player', 'position', 'location', 'shot_body_part',
       'shot_end_location', 'shot_first_time', 
         'shot_technique',
       'shot_type', 'type','shot_outcome']

In [17]:
Bundesliga_shots_df = pd.DataFrame(columns=relevant_attributes)

In [18]:
start = time.time()
for match in Bundesliga[Bundesliga['match_week']<=match_week_stage]['match_id']:
    match_df = sb.events(match_id=match)
    relevant_match_df = match_df[relevant_attributes][match_df['type']=='Shot']
    Bundesliga_shots_df = pd.concat([Bundesliga_shots_df, relevant_match_df], ignore_index=True)

Bundesliga_shots_df['shot_outcome'] = Bundesliga_shots_df['shot_outcome'].replace(
    to_replace={'Off T': 0, 'Blocked': 0, 'Saved': 0, 'Goal': 1, 'Wayward': 0, 'Post': 0,
               'Saved to Post': 0, 'Saved Off Target': 0})

Bundesliga_shots_df['distance'] = Bundesliga_shots_df.apply(lambda row: np.sqrt(np.abs(
    row.location[0]-120)**2 + np.abs(row.location[1]-40)**2), axis=1)
Bundesliga_shots_df['shot_angle'] = Bundesliga_shots_df.apply(
    lambda row: np.arctan2(np.abs(40-row.location[1]), np.abs(120-row.location[0])), axis=1)
end = time.time()
print(f'script run time: {(end - start)/60} minutes')
Bundesliga_shots_df

script run time: 4.822495563824972 minutes


,minute,second,team,shot_statsbomb_xg,player,position,location,shot_body_part,shot_end_location,shot_first_time,shot_technique,shot_type,type,shot_outcome,distance,shot_angle
0,4,21,Hoffenheim,0.087901,Tarik Elyounoussi,Left Wing Back,"[108.1, 31.2]",Left Foot,"[120.0, 35.2, 1.0]",True,Half Volley,Open Play,Shot,0,14.800338,0.636744
1,6,27,Schalke 04,0.160274,Klaas-Jan Huntelaar,Center Forward,"[110.9, 42.6]",Right Foot,"[120.0, 39.3, 0.5]",True,Volley,Open Play,Shot,1,9.464143,0.278300
2,8,16,Hoffenheim,0.016036,Kevin Volland,Left Attacking Midfield,"[117.9, 29.1]",Left Foot,"[118.1, 30.2]",True,Half Volley,Open Play,Shot,0,11.100450,1.380468
3,13,55,Schalke 04,0.527759,Jean-Eric Maxim Choupo-Moting,Left Midfield,"[101.8, 27.6]",Left Foot,"[120.0, 39.0, 0.2]",NaN,Normal,Open Play,Shot,1,22.022716,0.598078
4,17,16,Schalke 04,0.074020,Klaas-Jan Huntelaar,Center Forward,"[109.3, 26.5]",Right Foot,"[118.8, 36.0, 0.2]",NaN,Normal,Open Play,Shot,0,17.226143,0.900588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7826,72,14,Bayern Munich,0.248334,Thomas Müller,Right Center Midfield,"[113.6, 30.4]",Left Foot,"[120.0, 38.9, 0.2]",NaN,Normal,Open Play,Shot,1,11.537764,0.982794
7827,77,17,Bayern Munich,0.028587,David Olatukunbo Alaba,Left Back,"[99.2, 25.5]",Left Foot,"[117.7, 36.8, 2.2]",NaN,Normal,Open Play,Shot,0,25.355276,0.608787
7828,83,16,Bayern Munich,0.007272,Douglas Costa de Souza,Right Midfield,"[106.5, 64.6]",Left Foot,"[120.0, 37.3, 3.6]",NaN,Normal,Open Play,Shot,0,28.060827,1.068890
7829,86,15,Bayern Munich,0.025284,Douglas Costa de Souza,Right Midfield,"[100.7, 57.0]",Left Foot,"[120.0, 37.0, 0.7]",NaN,Normal,Open Play,Shot,1,25.719448,0.722122


## Describing the Data

In [19]:
Bundesliga_shots_df.describe()

,shot_statsbomb_xg,shot_outcome,distance,shot_angle
count,7831.000000,7831.000000,7831.000000,7831.000000
mean,0.106079,0.106883,18.857774,0.484138
std,0.149561,0.308984,8.526150,0.327077
min,0.000180,0.000000,0.943398,0.000000
25%,0.027162,0.000000,11.900420,0.213905
50%,0.054641,0.000000,17.924564,0.440802
75%,0.110142,0.000000,25.187298,0.709912
max,0.977242,1.000000,84.986881,1.570796


In [20]:
Bundesliga_shots_df = Bundesliga_shots_df[Bundesliga_shots_df.shot_type != 'Corner']

In [21]:
Bundesliga_shots_df['shot_outcome'].value_counts()

shot_outcome
0    6988
1     837
Name: count, dtype: int64

In [22]:
Bundesliga_team_summary_df = pd.DataFrame(columns = ['team', 'shots', 'goals', 'statsbomb_xg'])
for team in pd.unique(Bundesliga_shots_df['team']):
    shot_sum = Bundesliga_shots_df['team'].value_counts()[team]
    goal_sum = sum(Bundesliga_shots_df[Bundesliga_shots_df['team']==team]['shot_outcome'])
    statsbomb_xg_sum = sum(Bundesliga_shots_df[Bundesliga_shots_df['team']==team]['shot_statsbomb_xg'])
    xg_overperformance = goal_sum - statsbomb_xg_sum
    new_row = pd.Series({'team': team, 'shots': shot_sum, 'goals': goal_sum, 'statsbomb_xg': statsbomb_xg_sum, 'xg_overperformance': xg_overperformance})
    Bundesliga_team_summary_df.loc[len(Bundesliga_team_summary_df)]=new_row
Bundesliga_team_summary_df = Bundesliga_team_summary_df.sort_values(by=['goals'], ascending=False)
Bundesliga_team_summary_df

,team,shots,goals,statsbomb_xg
2,Bayern Munich,622,78,73.353521
9,Borussia Dortmund,524,80,77.142092
10,Wolfsburg,494,46,47.286714
11,VfB Stuttgart,490,48,53.967067
1,Schalke 04,486,47,45.795114
15,Bayer Leverkusen,479,51,49.798501
17,Borussia Mönchengladbach,457,64,49.197822
13,Werder Bremen,451,49,49.968887
12,Augsburg,439,40,41.013908
14,Ingolstadt,424,33,39.960963


In [23]:
Bundesliga_shots_df.columns

Index(['minute', 'second', 'team', 'shot_statsbomb_xg', 'player', 'position',
       'location', 'shot_body_part', 'shot_end_location', 'shot_first_time',
       'shot_technique', 'shot_type', 'type', 'shot_outcome', 'distance',
       'shot_angle'],
      dtype='object')

## Split training and test data, train model and define functions

In [24]:
cap_x_df, y_df = pd.concat([Bundesliga_shots_df.iloc[:,:-3], Bundesliga_shots_df.iloc[:,-2:]], axis=1),Bundesliga_shots_df.iloc[:,-3].to_frame()

In [25]:
train_cap_x_df, test_cap_x_df, train_y_df, test_y_df = \
train_test_split(cap_x_df, y_df,
                 test_size=0.20,
                random_state=42,
                 shuffle=True,
                stratify=y_df)

In [26]:
target_attr = 'shot_outcome'

In [27]:
numerical_attrs = ['distance', 'minute', 'shot_angle']

In [28]:
nominal_attrs = ['shot_technique', 'shot_type', 'shot_body_part', 'position', 'shot_first_time', 'position']

In [29]:
numerical_transformer = Pipeline(
        steps=[("imputer", SimpleImputer()),
               ("scaler", StandardScaler())]
    )

In [30]:
nominal_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="most_frequent")),
               #('target_encoder', TargetEncoder(target_type=target_type, random_state=42)),
               ("target_encoder", OrdinalEncoder()),
               ("scaler", StandardScaler())
               ]
    )

In [31]:
preprocessor = ColumnTransformer(
        transformers=[
            ('numerical', numerical_transformer, numerical_attrs),
            ('nominal', nominal_transformer, nominal_attrs)
        ]
    )

### The function below will be used to compare our models to the statsbomb model

In [ ]:
def plot_comparison(model1, model1_name, model2, model2_name, outcome):
    plt.figure()
    df = pd.DataFrame({'model1':model1,'model2':model2,'outcome':outcome})
    no_goal = df[df['outcome']==0]
    plt.scatter(no_goal['model1'],no_goal['model2'], label='no goal', color='blue', alpha=0.6)
    goal = df[df['outcome']==1]
    plt.scatter(goal['model1'],goal['model2'], label='goal', color='orange', alpha=1)
    #plt.scatter(model1, model2,c=outcome)
    plt.xlabel(model1_name)
    plt.ylabel(model2_name)
    plt.xlim(0,1.1)
    plt.ylim(0,1.1)
    plt.legend()
    z = np.polyfit(model1, model2,1)
    #z = np.polyfit(df['model1'], df['model2'],1)
    p = np.poly1d(z)
    plt.plot(model1, p(model1),"r--")
    #plt.plot(df['model1'], p(df['model1']),"r--")
    print(model1_name)
    print("y=%.2fx+%.2f"%(z[0],z[1]))
    print("$R^2$=", r2_score(model1,model2))

## Train our estimators

In [ ]:
estimator_names = ['LogisticRegression', 'SGDClassifier', 'RandomForestClassifier', 
                  'GradientBoostingClassifier', 'DecisionTreeClassifier']

estimator_list = [LogisticRegression(), SGDClassifier(loss='log_loss'), 
                  RandomForestClassifier(criterion='log_loss'), GradientBoostingClassifier(loss='log_loss'),
                 DecisionTreeClassifier(criterion='log_loss')]
trained_estimator_dict = {}
#model_summary = pd.DataFrame(columns = ['shots', 'goals', 'statsbomb_xg']
model_summary = {}
model_summary['shots'] = [len(train_y_df[target_attr])]
model_summary['goals'] = [train_y_df[target_attr].value_counts()[1]]
model_summary['statsbomb_xg'] = [sum(train_cap_x_df['shot_statsbomb_xg'])]
for estimator_name, estimator in zip(estimator_names, estimator_list):
    composite_estimator = Pipeline(steps=[('preprocessor', preprocessor), ('estimator', estimator)])
    trained_estimator_dict[estimator_name] = composite_estimator.fit(train_cap_x_df, train_y_df.values.ravel())
    predictions = trained_estimator_dict[estimator_name].predict_proba(train_cap_x_df)[:,1]
    SBModel = train_cap_x_df['shot_statsbomb_xg']
    shot_outcome = train_y_df[target_attr]
    model_summary[estimator_name] = [sum(predictions)]
    plot_comparison(predictions, estimator_name, SBModel, 'Statsbomb xG', shot_outcome)
#print(pd.DataFrame(model_summary))
pd.DataFrame.from_dict(model_summary)

## Select best model and examine permutation importance

In [ ]:
best_model_name = 'GradientBoostingClassifier'
best_model = trained_estimator_dict[best_model_name]

In [ ]:
feature_importances = permutation_importance(
    best_model, train_cap_x_df, train_y_df.values.ravel())

In [ ]:
mean_feature_importances = pd.Series(feature_importances.importances_mean, index=train_cap_x_df.columns)
mean_feature_importances.sort_values(ascending=False)

In [ ]:
std_feature_importances = pd.Series(feature_importances.importances_std, index=train_cap_x_df.columns)
std_feature_importances

In [ ]:
fig, ax = plt.subplots()
#mean_feature_importances[['distance','shot_body_part', 'shot_type', 'shot_angle']].plot.bar(
#    yerr=std_feature_importances[['distance','shot_body_part', 'shot_type', 'shot_angle']], ax=ax)
mean_feature_importances[numerical_attrs+nominal_attrs].plot.bar(
    yerr=std_feature_importances[numerical_attrs+nominal_attrs], ax=ax)
ax.set_title("Feature importances")
ax.set_ylabel("importance")
plt.xticks(rotation=45)
fig.tight_layout()
plt.show()

## Evaluate on Test Set

In [ ]:
test_predictions = best_model.predict_proba(test_cap_x_df)[:,1]
test_SBModel = test_cap_x_df['shot_statsbomb_xg']
test_shot_outcome = test_y_df[target_attr]
plot_comparison(test_predictions, best_model_name, test_SBModel, 'Statsbomb xG', test_shot_outcome)

## Observe behavior on totality of test set

In [ ]:
model_summary = pd.DataFrame(columns = ['shots', 'goals', 'statsbomb_xg', 'best_model_xg'])
shots = len(test_y_df[target_attr])
goals = test_y_df[target_attr].value_counts()[1]
statsbomb_xg = sum(test_cap_x_df['shot_statsbomb_xg'])
best_model_xg = sum(test_predictions)
new_row = pd.Series({'shots': shots, 'goals': goals, 'statsbomb_xg': statsbomb_xg, 'best_model_xg': best_model_xg})
model_summary.loc[len(model_summary)]=new_row
model_summary

## Examine on Week to Week Basis

### Get matches for a particular team

In [ ]:
def get_games_for_team(team):
    df = Bundesliga[(Bundesliga['home_team']==team)|(Bundesliga['away_team']==team)]
    df = df.sort_values(by='match_week')
    return df

In [ ]:
Dortmund_games = get_games_for_team('Borussia Dortmund')
Dortmund_games

## Examine individual matches
We'll look at a match after matchweek 25 since all the matches before matchweek 25 are what are used to train the model.

In [ ]:
next_game_week = 26

In [ ]:
match = Dortmund_games[Dortmund_games['match_week']==next_game_week]['match_id'].iloc[0]

In [ ]:
def cumulative_sums(array):
    return [sum(array[:i+1]) for i in range(len(array))]

In [ ]:
def display_match_results_with_xG(league_df,match, display_shots=False, display_chart=False):
    match_shots_df = pd.DataFrame(columns=relevant_attributes)
    #print(match)
    home_team = league_df[league_df['match_id']==match]['home_team'].values[0]
    home_score = league_df[league_df['match_id']==match]['home_score'].values[0]
    away_team = league_df[league_df['match_id']==match]['away_team'].values[0]
    away_score = league_df[league_df['match_id']==match]['away_score'].values[0]

    match_df = sb.events(match_id=match)
    new_match_df = match_df[relevant_attributes][match_df['type']=='Shot']

    match_shots_df = pd.concat([match_shots_df, new_match_df], ignore_index=True)

    match_shots_df['shot_outcome'] = match_shots_df['shot_outcome'].replace(
        to_replace={'Off T': 0, 'Blocked': 0, 'Saved': 0, 'Goal': 1, 'Wayward': 0, 'Post': 0,
                   'Saved to Post': 0, 'Saved Off Target': 0})

    match_shots_df['distance'] = match_shots_df.apply(lambda row: np.sqrt(np.abs(
        row.location[0]-120)**2 + np.abs(row.location[1]-40)**2), axis=1)
    match_shots_df['shot_angle'] = match_shots_df.apply(
        lambda row: np.arctan2(np.abs(40-row.location[1]), np.abs(120-row.location[0])), axis=1)
    match_shots_df = match_shots_df.sort_values(by='minute')
    
    cap_x_df = pd.concat([match_shots_df.iloc[:,:-3], 
                              match_shots_df.iloc[:,-2:]], axis=1)
    
    predictions = best_model.predict_proba(cap_x_df)[:,1]
    predictions_df = pd.DataFrame({'best_model': predictions})
    match_shots_df['best_model']= predictions_df
    
    home_shots_df = match_shots_df[match_shots_df['team']==home_team]
    away_shots_df = match_shots_df[match_shots_df['team']==away_team]

    home_cap_x_df = pd.concat([home_shots_df.iloc[:,:-4], 
                                home_shots_df.iloc[:,-3:]], axis=1)
    away_cap_x_df = pd.concat([away_shots_df.iloc[:,:-4], 
                                away_shots_df.iloc[:,-3:]], axis=1)
    
    home_predictions = home_cap_x_df['best_model']
    away_predictions = away_cap_x_df['best_model']
    
    home_total_best_model_xG = sum(home_predictions)
    away_total_best_model_xG = sum(away_predictions)
    home_statsbomb_xG = sum(home_cap_x_df['shot_statsbomb_xg'])
    away_statsbomb_xG = sum(away_cap_x_df['shot_statsbomb_xg'])
    if display_shots == True:
        display(match_shots_df)
    print("Proposed model")
    print("%s (%.2f) %.0f-%.0f (%.2f) %s" % (home_team, home_total_best_model_xG, home_score, away_score, away_total_best_model_xG, away_team))
    print("statsbomb xG")
    print("%s (%.2f) %.0f-%.0f (%.2f) %s" % (home_team, home_statsbomb_xG, home_score, away_score, away_statsbomb_xG, away_team))
    if display_chart==True:
        home_team_xg = [0]
        away_team_xg = [0]
        home_min = [0]
        away_min = [0]
        for x in range(len(match_shots_df)):
            if match_shots_df['team'][x] == home_team:
                home_team_xg.append(match_shots_df['best_model'][x])
                home_min.append(match_shots_df['minute'][x])
            if match_shots_df['team'][x] == away_team:
                away_team_xg.append(match_shots_df['best_model'][x])
                away_min.append(match_shots_df['minute'][x])
        home_cumulative_xg = cumulative_sums(home_team_xg)
        away_cumulative_xg = cumulative_sums(away_team_xg)

        fig, ax = plt.subplots(figsize= (10,5))
        plt.xticks([0,15,30,45,60,75,90])
        plt.xlabel("minute")
        plt.ylabel("model xG")
        plt.title("Cumulative xG by minute")
        home, = ax.step(x=home_min, y=home_cumulative_xg, label=home_team)
        away, = ax.step(x=away_min, y=away_cumulative_xg, label=away_team)
        ax.legend(handles=[home,away])
    #return match_shots_df

In [ ]:
display_match_results_with_xG(Dortmund_games,match, display_shots=False, display_chart=True)